## **Détection des valeurs aberrantes — Méthode Median (médiane ± k× MAD)**

In [ ]:
## CONFIGURATION ##

In [ ]:
# Set SNT Paths
SNT_ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(SNT_ROOT_PATH, "code")
CONFIG_PATH    <- file.path(SNT_ROOT_PATH, "configuration")

# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))

# List required packages 
required_packages <- c("dplyr", "tidyr", "terra", "ggplot2", "stringr", "lubridate", "viridis", "patchwork", "zoo", "scales", "purrr", "arrow", "sf", "reticulate", "knitr", "glue", "forcats")

# Execute function
install_and_load(required_packages)

# Set environment to load openhexa.sdk from the right environment
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

In [ ]:
# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

# Configuration variables
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_OUTLIERS_IMPUTATION
dataset_format_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED
indicator_defs <- config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
COUNTRY_NAME <- config_json$SNT_CONFIG$COUNTRY_NAME
ADM_1 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_1)
ADM_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)
facility_level <- config_json$SNT_CONFIG$ANALYTICS_ORG_UNITS_LEVEL

In [ ]:
# print function
printdim <- function(df, name = deparse(substitute(df))) {
  cat("Dimensions of", name, ":", nrow(df), "rows x", ncol(df), "columns\n\n")
}

In [ ]:
# import routine data
routine_data <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine_outliers-median_detection.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 analytics file for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

routine_data_imputed <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine_outliers-median_imputed.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 analytics file for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

shapes_data <- tryCatch({ get_latest_dataset_file_in_memory(dataset_format_name, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {                      
                      msg <- paste0(COUNTRY_NAME , " Shapes data is not available in dataset : " , dataset_format_name, " last version.")
                      log_msg(msg, "warning")
                      shapes_data <- NULL
                      })

printdim(routine_data)


### 1. Résumé des valeurs aberrantes détectées dans les données de routine

In [ ]:
outlier_summary_long <- routine_data %>%
  # select only the outlier-flag columns
  select(starts_with("OUTLIER_")) %>%
  # pivot to long
  pivot_longer(
    everything(),
    names_to = "method",
    values_to = "flag"
  ) %>%
  group_by(method) %>%
  summarise(
    n_outliers = sum(flag, na.rm = TRUE),
    pct_outliers = n_outliers / nrow(routine_data)
  ) %>%
  ungroup() %>%
  # make method names readable
  mutate(
    method_clean = method %>%
      str_replace("OUTLIER_", "") %>%
      str_replace("_", " ") %>%
      str_to_title()
  ) %>%
  select(Method = method_clean, n_outliers, pct_outliers)

outlier_summary_long

### 2. Visualisation des valeurs aberrantes (méthode Median ± MAD)

In [ ]:
#--- PARAMETERS ---
outlier_cols <- routine_data %>%
  select(starts_with("OUTLIER_MEDIAN")) %>%
  names()
print(outlier_cols)

In [ ]:
#--- FUNCTIONS TO MAKE ONE PLOT ---
plot_outliers <- function(ind_name, df, outlier_col) {
  
  df_ind <- df %>% filter(INDICATOR == ind_name)

  # Remove infinite or impossible values explicitly → removes warnings
  df_ind <- df_ind %>% 
    filter(!is.na(YEAR), !is.na(VALUE), is.finite(VALUE))

  p <- ggplot(df_ind, aes(x = YEAR, y = VALUE)) +
    
    # All values (grey)
    geom_point(alpha = 0.25, color = "grey40", na.rm = TRUE) +
    
    # Outliers (red)
    geom_point(
      data = df_ind %>% filter(.data[[outlier_col]] == TRUE),
      aes(x = YEAR, y = VALUE),
      color = "red",
      size = 2.8,
      alpha = 0.85,
      na.rm = TRUE
    ) +
    
    labs(
      title = paste("Inspection des valeurs aberrantes pour indicateur:", ind_name),
      subtitle = "Gris = toutes les valeurs • Rouge = valeurs aberrantes détectées",
      x = "Année",
      y = "Valeur"
    ) +
    theme_minimal(base_size = 14)

  return(p)
}

#plots <- map(unique_inds, ~ plot_outliers(.x, routine_data, outlier_col))
#walk(plots, print)

plot_outliers_by_district_facet_year <- function(ind_name, df, outlier_col) {
  
  df_ind <- df %>%
    filter(
      INDICATOR == ind_name,
      !is.na(YEAR),
      !is.na(VALUE),
      is.finite(VALUE)
    )
  
  if (nrow(df_ind) == 0) return(NULL)
  
  ggplot(df_ind, aes(x = ADM2_ID, y = VALUE)) +
    geom_point(color = "grey60", alpha = 0.3) +
    geom_point(
      data = df_ind %>% filter(.data[[outlier_col]] == TRUE),
      color = "red", 
      size = 2.8,
      alpha = 0.85
    ) +
    facet_wrap(~ YEAR, scales = "free_y") +
    labs(
      title = paste("Détection des valeurs aberrantes —", ind_name),
      subtitle = paste("Méthode :", outlier_col, "| Rouge = valeur aberrante"),
      x = "District (ADM2)",
      y = "Valeur"
    ) +
    theme_minimal(base_size = 13) +
    theme(
      axis.text.x = element_text(angle = 75, hjust = 1, size = 7)
    )
}

### Include plots  

-Clean folder  
-Save Images  
-Load the images  

In [ ]:
# Create folder if it doesn't exist
output_dir <- file.path(getwd(), "outputs/plots")
if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE) 
} else {  
  files <- list.files(output_dir, full.names = TRUE)
  if (length(files) > 0) file.remove(files)
}

In [ ]:
selected_inds <- c("SUSP", "TEST", "CONF")

# Faster and safer for memory 
for (col in outlier_cols) {
  for (ind in selected_inds) {
    p <- plot_outliers_by_district_facet_year(ind, routine_data, col)    
    if (!is.null(p)) {      
      file_name <- file.path(output_dir, paste0("outliers_", col, "_", ind, ".png")) 
      ggsave(filename = file_name, plot = p, width = 14, height = 8, dpi = 150)  # Save plot as PNG            
      rm(p)
      gc()
    }
  }
}

# List all PNG files
img_files <- sort(list.files(path = output_dir, pattern = "\\.png$", full.names = TRUE))
for (img in img_files) {
  IRdisplay::display_png(file = img)
}

### 3. Cohérence des indicateurs au niveau nationale

La section ci-dessous est un extrait des explications fournies par la **Community code library for SNT**. Veuillez consulter le site Web pour obtenir des explications complètes: https://ahadi-analytics.github.io/snt-code-library/english/library/data/routine_cases/quality_control.html#cb19-55

**Consultations externes toutes causes confondues ≥ cas suspects de paludisme**: à tout moment dans un établissement de santé, le nombre de consultations externes toutes causes confondues doit toujours être supérieur à toute autre variable déclarée à ce moment-là par cet établissement, car le nombre de consultations externes est toujours le point de départ d'un processus clinique. Un nombre de consultations externes toutes causes confondues inférieur au nombre de cas suspects de paludisme est généralement révélateur d'une mauvaise qualité des informations non spécifiques au paludisme.

**Cas suspects de paludisme ≥ cas de paludisme testés**: la recommandation générale pour la prise en charge des cas de paludisme est que tous les patients ambulatoires présentant de la fièvre (voir la définition des cas suspects dans les directives nationales de prise en charge des cas de paludisme) doivent subir un test de dépistage du paludisme. Par conséquent, dans l'idéal, le nombre de cas suspects de paludisme devrait être égal au nombre de cas de paludisme testés.

**Consultations externes toutes causes confondues ≥ cas de paludisme testés**: cette vérification peut être intéressante si le NMP suggère que les informations contenues dans la colonne « cas suspects de paludisme » ne sont pas fiables et que la vérification précédente ne peut être effectuée.

**Cas de paludisme testés ≥ cas de paludisme confirmés**: les points représentent les taux de positivité des tests (TPR) au niveau des établissements : TPR = cas de paludisme confirmés divisés par les cas de paludisme testés. On s'attend à ce qu'il n'y ait pratiquement aucune situation où les taux de positivité des tests seront de 100 %. En outre, il n'est pas possible d'avoir un nombre de cas confirmés supérieur au nombre de cas testés (TPR > 100 %), sauf dans des circonstances très limitées, telles que les centres de santé de référence pour d'autres établissements de santé ou les agents de santé communautaires (à discuter avec l'équipe SNT). Si le nombre de cas confirmés est supérieur au nombre de cas testés, il y a probablement un problème de qualité des données, soit au niveau d'un élément de données, soit au niveau des deux.

**Cas de paludisme confirmés ≥ cas de paludisme traités**: dans l'idéal, tous les cas confirmés devraient être traités avec la dose thérapeutique adaptée à l'âge des traitements de première intention dans le pays. Par conséquent, le nombre de cas de paludisme confirmés devrait être le même que le nombre de cas de paludisme traités. On observe de faibles taux de traitement lorsque les cas confirmés sont plus nombreux que les cas traités, ce qui peut être dû à des ruptures de stock d'ACT ou à des problèmes de qualité des données. Un nombre de cas confirmés inférieur au nombre de cas traités peut indiquer l'existence de traitements présomptifs, la vente de médicaments provenant des établissements de santé dans le secteur privé ou des problèmes liés à la qualité des données. Il peut être utile de vérifier les données relatives aux stocks des établissements individuels présentant des taux de traitement systématiquement élevés ou faibles.

**Admissions à l'hôpital toutes causes confondues ≥ admissions pour paludisme**: le nombre de patients hospitalisés pour paludisme ne peut être supérieur au nombre total de patients hospitalisés. Lorsque les admissions toutes causes confondues sont inférieures aux admissions pour paludisme, cela reflète des problèmes de qualité des données liés à une déclaration inadéquate de l'une ou l'autre des variables.

**Décès toutes causes confondues ≥ décès dus au paludisme**: un rapport de 1:1 implique que tous les décès déclarés sont attribués au paludisme. Lorsque les décès toutes causes confondues sont inférieurs aux décès dus au paludisme, cela reflète des problèmes de qualité des données liés à une déclaration inadéquate de l'une ou des deux variables.

**Admissions pour paludisme ≥ décès dus au paludisme**: un rapport de 1:1 implique que tous les cas de paludisme hospitalisés sont décédés, soit un taux de mortalité hospitalière de 100 %. Lorsque les admissions pour paludisme sont inférieures aux décès dus au paludisme, cela reflète des problèmes de qualité des données liés à une déclaration inadéquate de l'une ou des deux variables.

In [ ]:
# Step 1: Extract year, month from PERIOD & aggregate
routine_month <- routine_data_imputed %>%
  mutate(
    YEAR  = substr(PERIOD, 1, 4),
    MONTH = substr(PERIOD, 5, 6),
    DATE  = as.Date(paste0(YEAR, "-", MONTH, "-01"))
  ) %>%
  group_by(YEAR, MONTH, DATE) %>%
  summarise(
    SUSP    = sum(SUSP,    na.rm = TRUE),
    TEST    = sum(TEST,    na.rm = TRUE),
    CONF    = sum(CONF,    na.rm = TRUE),
    PRES    = sum(PRES,    na.rm = TRUE),
    .groups = "drop"
  )

In [ ]:
# Step 2: Plot monthly national trends
options(repr.plot.width = 14, repr.plot.height = 6)
routine_month %>%
  pivot_longer(cols = c(SUSP, TEST, CONF, PRES), names_to = "Indicator") %>%
  ggplot(aes(x = DATE, y = value, color = Indicator)) +
  geom_line(linewidth = 1.2) +
  labs(
    title = "Tendances mensuelles nationales des indicateurs composites (après suppression des outliers)",
    x = "Mois", y = "Nombre de cas", color = "Indicateur"
  ) +
  theme_minimal(base_size = 16) +
  theme(
    plot.title = element_text(face = "bold", size = 20),
    axis.title = element_text(size = 16),
    axis.text = element_text(size = 16),
    legend.title = element_text(size = 16),
    legend.text = element_text(size = 16)
  )

In [ ]:
# Identify indicator columns automatically (all numeric except YEAR, MONTH, IDs)
indicator_cols <- routine_data_imputed %>%
  select(where(is.numeric)) %>%
  select(-PERIOD, -YEAR, -MONTH) %>% 
  colnames()

yearly_totals <- routine_data_imputed %>%
  group_by(YEAR) %>%
  summarise(across(all_of(indicator_cols), ~ sum(.x, na.rm = TRUE))) %>%
  ungroup()

yearly_totals %>% select(YEAR, SUSP, TEST, CONF)

In [ ]:
# Step 3: Create scatter plots
routine_hd_month <- routine_data_imputed %>%
  mutate(
    YEAR  = substr(PERIOD, 1, 4),
    MONTH = substr(PERIOD, 5, 6),
    DATE  = as.Date(paste0(YEAR, "-", MONTH, "-01"))   
  ) %>%
  group_by(ADM2_ID, YEAR, MONTH, DATE) %>%
  summarise(
    SUSP    = sum(SUSP,    na.rm = TRUE),
    TEST    = sum(TEST,    na.rm = TRUE),
    CONF    = sum(CONF,    na.rm = TRUE),
    PRES    = sum(PRES,    na.rm = TRUE),
    MALTREAT    = sum(MALTREAT,    na.rm = TRUE),
    .groups = "drop"
  )

options(repr.plot.width = 14, repr.plot.height = 6)

p1 <- ggplot(routine_hd_month, aes(x = SUSP, y = TEST)) +
  geom_point(alpha = 0.5, color = "blue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Suspectés vs Testés", x = "Cas suspectés", y = "Cas testés") +
  theme_minimal(base_size = 16)

p2 <- ggplot(routine_hd_month, aes(x = TEST, y = CONF)) +
  geom_point(alpha = 0.5, color = "darkgreen") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Testés vs Confirmés", x = "Cas testés", y = "Cas confirmés") +
  theme_minimal(base_size = 16)

p3 <- ggplot(routine_hd_month, aes(x = CONF, y = MALTREAT)) +
  geom_point(alpha = 0.5, color = "purple") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Confirmés vs Traités", x = "Cas confirmés", y = "Cas traités") +
  theme_minimal(base_size = 16)

# Step 3: Combine plots
(p1 | p2 | p3) + plot_layout(guides = "collect")

Le graphique en bas montre le **pourcentage de rapports mensuels des formations sanitaires au niveau national** qui ont passé chaque contrôle de cohérence pour chaque année. Chaque cellule indique la proportion de rapports mensuels d’une année donnée qui respectent la règle de cohérence correspondante. Évaluer ces contrôles d’une année à l’autre et entre catégories permet d’identifier les **tendances générales de la qualité des données**.

In [ ]:
# ---- 0. Define the checks, columns and labels ----
checks <- list(
  allout_susp = c("ALLOUT", "SUSP"),       
  allout_test = c("ALLOUT", "TEST"),       
  susp_test   = c("SUSP", "TEST"),         
  test_conf   = c("TEST", "CONF"),         
  conf_treat  = c("CONF", "MALTREAT"),     
  adm_dth     = c("MALADM", "MALDTH")      
)

check_labels <- c(
  pct_coherent_allout_susp = "Ambulatoire ≥ Suspects",
  pct_coherent_allout_test = "Ambulatoire ≥ Testés",
  pct_coherent_susp_test   = "Suspects ≥ Testés",
  pct_coherent_test_conf   = "Testés ≥ Confirmés",
  pct_coherent_conf_treat  = "Confirmés ≥ Traités",
  pct_coherent_adm_dth     = "Admissions Palu ≥ Décès Palu"
)

In [ ]:
df <- routine_data_imputed

# ---- 1. Build coherency checks dynamically ----
df_checks <- df %>%
  mutate(
    !!!lapply(names(checks), function(check_name) {
      cols <- checks[[check_name]]
      if (all(cols %in% names(df))) {
        expr(!!sym(cols[1]) >= !!sym(cols[2]))
      } else {
        expr(NA)
      }
    }) %>% setNames(paste0("check_", names(checks)))
  )

# ---- 2. Summarise percent coherent per year ----
check_cols <- intersect(paste0("check_", names(checks)), names(df_checks))

coherency_metrics <- df_checks %>%
  group_by(YEAR) %>%
  summarise(
    across(all_of(check_cols), ~ mean(.x, na.rm = TRUE) * 100,
           .names = "pct_{.col}"),
    .groups = "drop"
  ) %>%
  pivot_longer(
    cols = starts_with("pct_"),
    names_to = "check_type",
    names_prefix = "pct_check_",
    values_to = "pct_coherent"
  ) %>%
  filter(!is.na(pct_coherent)) %>%  # <-- remove missing checks entirely
  mutate(
    check_label = recode(
      check_type,
      !!!setNames(check_labels, sub("^pct_coherent_", "", names(check_labels)))
    ),
    check_label = factor(check_label, levels = unique(check_label)),  # preserve only existing levels
    check_label = fct_reorder(check_label, pct_coherent, .fun = median, na.rm = TRUE)
  )

# ---- 3. Heatmap ----
coherency_plot <- ggplot(coherency_metrics, aes(
  x = factor(YEAR),
  y = check_label,
  fill = pct_coherent
)) +
  geom_tile(color = NA, width = 0.88, height = 0.88) +
  geom_text(
    aes(label = sprintf("%.0f%%", pct_coherent)),
    color = "white",
    fontface = "bold",
    size = 5
  ) +
  scale_fill_viridis(
    name = "% Cohérent",
    option = "viridis",
    limits = c(0, 100),
    direction = -1
  ) +
  labs(
    title = "Contrôles de cohérence des données (niveau national)",
    x = "Année",
    y = NULL
  ) +
  theme_minimal(base_size = 14) +
  theme(
    panel.grid = element_blank(),
    plot.title = element_text(size = 22, face = "bold", hjust = 0.5),
    axis.text.y = element_text(size = 16, hjust = 0),
    axis.text.x = element_text(size = 16),
    legend.title = element_text(size = 16, face = "bold"),
    legend.text = element_text(size = 14),
    legend.key.width = unit(0.7, "cm"),
    legend.key.height = unit(1.2, "cm")
  )

coherency_plot

### 4. Visualisation de la cohérence au niveau du AMD1

In [ ]:
df <- routine_data_imputed

# ---- 1. Build coherency check per row safely ----
df_checks <- df %>%
  mutate(
    !!!lapply(names(checks), function(check_name) {
      cols <- checks[[check_name]]
      if (all(cols %in% names(df))) {
        expr(!!sym(cols[1]) >= !!sym(cols[2]))
      } else {
        expr(NA_real_)
      }
    }) %>% setNames(paste0("check_", names(checks)))
  )

# Identify the check columns that actually exist
check_cols <- names(df_checks)[grepl("^check_", names(df_checks))]

valid_checks <- check_cols[
  purrr::map_lgl(df_checks[check_cols], ~ !all(is.na(.x)))
]

# Compute coherence
adm_coherence <- df_checks %>%
  group_by(ADM1_NAME, ADM2_NAME, ADM2_ID, YEAR) %>%
  summarise(
    total_reports = n(),
    !!!purrr::map(
      valid_checks,
      ~ expr(100 * mean(.data[[.x]], na.rm = TRUE))
    ) %>%
      setNames(paste0("pct_coherent_", sub("^check_", "", valid_checks))),
    .groups = "drop"
  ) %>%
  filter(total_reports >= 5)

# To long format
adm_long <- adm_coherence %>%
  pivot_longer(
    cols = starts_with("pct_coherent_"),
    names_to  = "check_type",
    values_to = "pct_coherent"
  ) %>%
  filter(!is.na(pct_coherent))

adm_long <- adm_long %>% mutate(check_label = recode(check_type, !!!check_labels))

head(adm_long)

In [ ]:
# Define heatmap function
plot_coherence_heatmap <- function(df, selected_year, agg_level = "ADM1_NAME", filename = NULL, do_plot = TRUE) {
  
  if (!agg_level %in% names(df)) {
    stop(paste0("Aggregation level '", agg_level, "' not found in data!"))
  }
  
  # Aggregate pct_coherent by chosen level + check_label
  df_year <- df %>%
    filter(YEAR == selected_year) %>%
    group_by(across(all_of(c(agg_level, "check_label")))) %>%
    summarise(
      pct_coherent = mean(pct_coherent, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    group_by(across(all_of(agg_level))) %>%
    mutate(median_coh = median(pct_coherent, na.rm = TRUE)) %>%
    ungroup() %>%
    mutate(!!agg_level := fct_reorder(.data[[agg_level]], median_coh))
  
  n_units <- n_distinct(df_year[[agg_level]])
  plot_height <- max(6, 0.5 * n_units)  # dynamically adjust height
  agg_label <- if (agg_level == "ADM1_NAME") {
      "niveau administratif 1"
    } else if (agg_level == "ADM2_NAME") {
      "niveau administratif 2"
    } else {
      agg_level  # fallback, in case a different level is passed
    }
    
  p <- ggplot(df_year, aes(x = check_label, y = .data[[agg_level]], fill = pct_coherent)) +
    geom_tile(color = "white", linewidth = 0.2) +
    geom_text(aes(label = sprintf("%.0f%%", pct_coherent)),
              size = 5, fontface = "bold", color = "white") +
    scale_fill_viridis(name = "% cohérent", limits = c(0, 100),
                       option = "viridis", direction = -1) +
    labs(
      title = paste0("Cohérence des données par ", agg_label, " - ", selected_year),
      x = "Règle de cohérence",
      y = agg_label
    ) +
    theme_minimal(base_size = 14) +
    theme(
      panel.grid = element_blank(),
      axis.text.y = element_text(size = 12),
      axis.text.x = element_text(size = 12, angle = 30, hjust = 1),
      plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
      legend.title = element_text(size = 12),
      legend.text = element_text(size = 10)
    )
  
  # Adjust notebook display
  options(repr.plot.width = 14, repr.plot.height = plot_height)
  
  # Save if filename is provided
  if (!is.null(filename)) {
    ggsave(filename = filename, plot = p,
           width = 14, height = plot_height, dpi = 300,
          limitsize = FALSE)
  }
  if (do_plot) { print(p) }
  # return(p)
}

In [ ]:
# Plot per year
years_available <- sort(unique(adm_long$YEAR))
for (year in years_available) {
    plot_coherence_heatmap(df = adm_long, selected_year = year, agg_level = "ADM1_NAME")
}

### 5. Visualisation de la cohérence au niveau du AMD2

In [ ]:
shapes_data <- shapes_data %>%
  mutate(ADM2_ID = as.character(ADM2_ID))

adm_coherence <- adm_coherence %>%
  mutate(ADM2_ID = as.character(ADM2_ID))

map_data <- shapes_data %>%
  left_join(adm_coherence, by = "ADM2_ID")

map_data <- map_data %>%
  rename(
    ADM2_NAME_shape = ADM2_NAME.x,
    ADM2_NAME_data  = ADM2_NAME.y
  )

In [ ]:
# Define function
plot_coherence_map <- function(map_data, col_name, indicator_label = NULL) {
  
  # Check if column exists
  if (!col_name %in% names(map_data)) {
    stop(paste0("Column '", col_name, "' not found in the data!"))
  }
  
  # Default legend title if not provided
  if (is.null(indicator_label)) {
    indicator_label <- col_name
  }
  
  ggplot(map_data) +
    geom_sf(aes(fill = .data[[col_name]]), color = "white", size = 0.2) +
    scale_fill_viridis(
      name = paste0("% cohérence\n(", indicator_label, ")"),
      option = "magma",
      direction = -1,
      limits = c(0, 100),
      na.value = "grey90"
    ) +
    # facet_wrap(~ YEAR) +
    facet_wrap(~ YEAR, drop = TRUE) +
    labs(
      title = "Cohérence des données par niveau administratif 2 et par année",
      subtitle = paste("Indicateur :", indicator_label),
      caption = "Source : DHIS2 données routinières"
    ) +
    theme_minimal(base_size = 15) +
    theme(
      panel.grid = element_blank(),
      strip.text = element_text(size = 14, face = "bold"),
      plot.title = element_text(size = 20, face = "bold"),
      legend.position = "right"
    )
}


In [ ]:
# Loop over all available columns
for (check_col in names(check_labels)) {
  
  # Only proceed if the column actually exists in the dataframe
  if (check_col %in% names(map_data)) {        
    label <- check_labels[[check_col]]        
    p <- plot_coherence_map(map_data, col_name = check_col, indicator_label = label)        
    print(p)
    
    # Optionally, save the plot to a file
    # ggsave(filename = paste0("heatmap_", check_col, ".png"), plot = p,
    #        width = 14, height = 10, dpi = 300)
  }
}